In [9]:
import gensim
from gensim import corpora
from gensim.models import Word2Vec

In [10]:
import gensim
print(gensim.__version__)

4.3.0


In [11]:
import numpy as np
import pandas as pd

In [12]:
temp_df = pd.read_csv("IMDB Dataset.xls")
temp_df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [13]:
df = temp_df.iloc[:10000]

In [14]:
df.drop_duplicates(inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_11820\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [15]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile('<,*?>'), '',raw_text)
    return cleaned_text

In [16]:
df['review']=df['review'].apply(remove_tags)

C:\Users\HP\AppData\Local\Temp\ipykernel_11820\2928534252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_tags)


In [17]:
df['review']=df['review'].apply(lambda x:x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_11820\2697927567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x:x.lower())


In [18]:
from nltk.corpus import stopwords
sw_list=stopwords.words('english')
df['review']=df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_11820\2738062930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [19]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. <br /><br />the f...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [20]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [21]:
story=[]
for doc in df['review']:
    raw_sent=sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [22]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [23]:
model.build_vocab(story)

In [24]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5936092, 6424740)

In [25]:
len(model.wv.index_to_key)

31846

In [26]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc=[word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [27]:
document_vector(df['review'].values[0])

array([-0.16706185,  0.35965943,  0.03894396,  0.01937195,  0.12901554,
       -0.6659902 ,  0.20168662,  0.8519823 , -0.06747704, -0.23905136,
       -0.04125547, -0.57372326,  0.0189331 ,  0.2813121 ,  0.03207425,
       -0.18095694,  0.26299202, -0.3089436 ,  0.00511205, -0.7009649 ,
       -0.11170226,  0.10761684,  0.29364148, -0.00333023,  0.03230974,
        0.03595071, -0.3123192 , -0.08550986, -0.39398134, -0.01130691,
        0.3693187 ,  0.06210081,  0.22144374, -0.47339234, -0.17231087,
        0.47804302,  0.05533222, -0.2711159 , -0.30510142, -0.8199247 ,
       -0.17645203, -0.14058545, -0.14125423,  0.08657725,  0.593793  ,
       -0.1594115 , -0.30085775, -0.15424618,  0.11381534,  0.11475222,
        0.10430777, -0.25519842, -0.32515773, -0.04085213, -0.13094874,
        0.09059025,  0.29133332, -0.06062601, -0.43643683,  0.11887036,
        0.05506122,  0.04677002, -0.03335215,  0.12861277, -0.5044098 ,
        0.2916519 ,  0.11171313,  0.20112707, -0.47389355,  0.47

In [28]:
document_vector(df['review'].values[0])

array([-0.16706185,  0.35965943,  0.03894396,  0.01937195,  0.12901554,
       -0.6659902 ,  0.20168662,  0.8519823 , -0.06747704, -0.23905136,
       -0.04125547, -0.57372326,  0.0189331 ,  0.2813121 ,  0.03207425,
       -0.18095694,  0.26299202, -0.3089436 ,  0.00511205, -0.7009649 ,
       -0.11170226,  0.10761684,  0.29364148, -0.00333023,  0.03230974,
        0.03595071, -0.3123192 , -0.08550986, -0.39398134, -0.01130691,
        0.3693187 ,  0.06210081,  0.22144374, -0.47339234, -0.17231087,
        0.47804302,  0.05533222, -0.2711159 , -0.30510142, -0.8199247 ,
       -0.17645203, -0.14058545, -0.14125423,  0.08657725,  0.593793  ,
       -0.1594115 , -0.30085775, -0.15424618,  0.11381534,  0.11475222,
        0.10430777, -0.25519842, -0.32515773, -0.04085213, -0.13094874,
        0.09059025,  0.29133332, -0.06062601, -0.43643683,  0.11887036,
        0.05506122,  0.04677002, -0.03335215,  0.12861277, -0.5044098 ,
        0.2916519 ,  0.11171313,  0.20112707, -0.47389355,  0.47

In [29]:
from tqdm import tqdm

In [30]:
X=[]
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [28:06<00:00,  5.92it/s]


In [32]:
X=np.array(X)

In [33]:
X[0]

array([-0.16706185,  0.35965943,  0.03894396,  0.01937195,  0.12901554,
       -0.6659902 ,  0.20168662,  0.8519823 , -0.06747704, -0.23905136,
       -0.04125547, -0.57372326,  0.0189331 ,  0.2813121 ,  0.03207425,
       -0.18095694,  0.26299202, -0.3089436 ,  0.00511205, -0.7009649 ,
       -0.11170226,  0.10761684,  0.29364148, -0.00333023,  0.03230974,
        0.03595071, -0.3123192 , -0.08550986, -0.39398134, -0.01130691,
        0.3693187 ,  0.06210081,  0.22144374, -0.47339234, -0.17231087,
        0.47804302,  0.05533222, -0.2711159 , -0.30510142, -0.8199247 ,
       -0.17645203, -0.14058545, -0.14125423,  0.08657725,  0.593793  ,
       -0.1594115 , -0.30085775, -0.15424618,  0.11381534,  0.11475222,
        0.10430777, -0.25519842, -0.32515773, -0.04085213, -0.13094874,
        0.09059025,  0.29133332, -0.06062601, -0.43643683,  0.11887036,
        0.05506122,  0.04677002, -0.03335215,  0.12861277, -0.5044098 ,
        0.2916519 ,  0.11171313,  0.20112707, -0.47389355,  0.47

In [34]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

y=encoder.fit_transform(df['sentiment'])

In [35]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [38]:
rf=RandomForestClassifier
Classifier=RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
Classifier.fit(X_train, y_train)
y_pred=Classifier.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_test, y_pred)
print(f"Accuracy:{accuracy:2f}")